In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import math
import time
import pickle  #Para guardar archivos
import os
os.environ['OMP_NUM_THREADS'] = '1'

from pympler import asizeof #Para liberar memoria
import gc

from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

import scipy.cluster.hierarchy as hierarchy
import scipy.spatial.distance as distance
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage

from sklearn.neighbors import kneighbors_graph
from scipy.interpolate import interp1d
from sklearn.cluster import DBSCAN

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import neighbors

from sklearn.metrics import accuracy_score, precision_score,recall_score,classification_report,confusion_matrix,mean_absolute_error

import joblib

semilla = 8

In [2]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
    r_ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta =   "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
    r_ruta = "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='DESKTOP-11LG84H':
    ruta = "C:/Users/MyHP/Desktop/VIU_TFM"
    r_ruta = "C:/Users/MyHP/Desktop/VIU_TFM"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [3]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = ruta  
nombre_archivo = 'TABLA_AFI_MAESTRIA_2.dsv'
ruta_archivo = os.path.join(directorio, nombre_archivo)

data_l = pd.read_csv(ruta_archivo, delimiter=';')  

In [4]:
print(data_l.head())
print(data_l.columns)

   IDENTIFICADOR SEXO FECHA_NACIMIENTO  BASE_CALCULO_2022  IMPO_2022  \
0       15277464    H       28/08/1980           958.2655        175   
1         751627    H       26/09/2001           404.7500         39   
2        1018328    H       16/08/1970           135.6177         16   
3        1608824    M       02/02/1985          1428.3333        144   
4        1447852    H       29/02/1988          1435.5716        132   

    SAL_PROM_1     SUELDO   SAL_PROM_2     Q1_1    Q3_1  ...  N_PRI  N_PUB  \
0   754.643785  133571.95   754.643785   735.00   944.1  ...    2.0  161.0   
1   407.230769   15882.00   407.230769   400.00   425.0  ...    NaN    NaN   
2   175.643889    3161.59   373.646250    16.32   425.0  ...    7.0    NaN   
3  1272.388889  183224.00  1272.388889  1212.00  1412.0  ...    NaN  144.0   
4  1026.027143  136461.61  1026.027143   469.36  1212.0  ...   19.0  112.0   

  N_IND N_VOL_EC  N_VOL_EX  M_PRI  M_PUB  M_IND  M_VOL_EC  M_VOL_EX  
0   NaN      NaN       NaN  

In [12]:
#data_l = data_l.head(500000)

In [5]:
data_l.shape

(3089379, 26)

In [84]:
# data_l = data_l[~data_l['FECHA_NACIMIENTO'].isin(['01/01/0056', '01/01/0083', '09/10/0057', '01/01/0052','01/01/0061',
#                                                  '01/01/0050', '01/01/0053','01/01/0059', '01/01/0059','01/01/0082',
#                                                   '01/01/0040','01/01/0036', '05/02/0059', '01/01/0022', '01/01/0085',
#                                                  '01/01/0058', '01/12/0026', '01/12/0026', '01/01/0067'])]

In [6]:
data_l = data_l[~data_l['FECHA_NACIMIENTO'].str.contains(r'^00', na=False)]
data_l['FECHA_NACIMIENTO'] = pd.to_datetime(data_l['FECHA_NACIMIENTO'], format='%d/%m/%Y', errors='coerce')

In [7]:
# Creación de las variables de los límites superiores
data_l['LS_MS'] = data_l['T3_1'] + 1.5*(data_l['T3_1']-data_l['T1_1'])
data_l['LS1'] = data_l['Q3_1'] + 1.5*(data_l['Q3_1']-data_l['Q1_1'])
data_l['LS2'] = data_l['Q3_2'] + 1.5*(data_l['Q3_2']-data_l['Q1_2'])

In [8]:
# Corrección de variables
data_l= data_l[data_l['SEXO'].notna()]
data_l['BASE_CAL'] = data_l['BASE_CALCULO_2022']
data_l['NUMERO_IMPOSICIONES'] = data_l['IMPO_2022']

In [9]:
# Tranformación de variables
lb = preprocessing.LabelBinarizer()
data_l['SEXO_T'] =data_l['SEXO']
data_l['SEXO'] = lb.fit_transform( data_l['SEXO'])

data_l['RANGO_INI_5MEJ'] = data_l['INICIO_5MEJ']

data_l['RANGO_FIN_5MEJ'] = data_l['FIN_5MEJ']

data_l['LS2'] = np.where( data_l['LS2'].isna(),
                         data_l['LS1'],
                         data_l['LS2'] )

data_l['SAL_PROM2'] = np.where( data_l['SAL_PROM_2'].isna(),
                               data_l['SAL_PROM_1'],
                               data_l['SAL_PROM_2'] )
data_l['PRES'] = 2
data_l['FECHA_NACIMIENTO'] = pd.to_datetime(data_l['FECHA_NACIMIENTO'], format='%d/%m/%Y')
data_l['INICIO_5MEJ'] = pd.to_datetime(data_l['INICIO_5MEJ'], format='%d/%m/%Y')
data_l['FIN_5MEJ'] = pd.to_datetime(data_l['FIN_5MEJ'], format='%d/%m/%Y')

In [10]:
today = datetime.today()

fecha_objetivo = pd.to_datetime('31-12-2023', format='%d-%m-%Y')
data_l['EDAD_J'] = (fecha_objetivo - data_l['FECHA_NACIMIENTO']).dt.days // 365
data_l['TIEM_T'] = data_l['FECHA_NACIMIENTO'].apply(lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)))
data_l['TIEM_MA'] = (data_l['INICIO_5MEJ'] - data_l['FIN_5MEJ']).dt.days // 30
data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX',
        'N_PRI', 'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']]= data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX','N_PRI', 
                              'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']].fillna(0)
data_l['M_PRI_P'] = data_l['M_PRI'] / data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX']].sum(axis=1)
data_l['M_PUB_P'] = data_l['M_PUB'] / data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX']].sum(axis=1)
data_l['M_IND_P'] = data_l['M_IND'] / data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX']].sum(axis=1)
data_l['M_VOL_EC_P'] = data_l['M_VOL_EC'] / data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX']].sum(axis=1)
data_l['M_VOL_EX_P'] = data_l['M_VOL_EX'] / data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX']].sum(axis=1)

data_l['N_PRI_P'] = data_l['N_PRI'] / data_l[['N_PRI', 'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']].sum(axis=1)
data_l['N_PUB_P'] = data_l['N_PUB'] / data_l[['N_PRI', 'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']].sum(axis=1)
data_l['N_IND_P'] = data_l['N_IND'] / data_l[['N_PRI', 'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']].sum(axis=1)
data_l['N_VOL_EC_P'] = data_l['N_VOL_EC'] / data_l[['N_PRI', 'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']].sum(axis=1)
data_l['N_VOL_EX_P'] = data_l['N_VOL_EX'] / data_l[['N_PRI', 'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']].sum(axis=1)

In [19]:
r_ruta

'E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/RData/TFM/'

In [11]:
ruta1 = r_ruta + 'XGBClassifier_random_search.joblib'
modelo = joblib.load(ruta1)

In [12]:
#Selección de variables para entrenar los modelos
val_otr = ['BASE_CAL', 'LS2', 'SAL_PROM2', 'LS_MS', 'SUELDO',
           'SEXO', 'PRES', 'N_PRI_P', 'N_PUB_P', 'N_IND_P', 'N_VOL_EC_P',
           'N_VOL_EX_P', 'M_PRI_P', 'M_PUB_P', 'M_IND_P', 'M_VOL_EC_P','M_VOL_EX_P',
           'NUMERO_IMPOSICIONES', 'TIEM_T']
data_otr = data_l[ val_otr ]
data_otr

,BASE_CAL,LS2,SAL_PROM2,LS_MS,SUELDO,SEXO,PRES,N_PRI_P,N_PUB_P,N_IND_P,N_VOL_EC_P,N_VOL_EX_P,M_PRI_P,M_PUB_P,M_IND_P,M_VOL_EC_P,M_VOL_EX_P,NUMERO_IMPOSICIONES,TIEM_T
0,958.2655,1257.75,754.643785,1124.71375,133571.95,0,2,0.012270,0.987730,0.0,0.000000,0.0,0.0,1.000000,0.0,0.000000,0.0,175,44
1,404.7500,462.50,407.230769,462.50000,15882.00,0,2,0.000000,0.000000,0.0,1.000000,0.0,0.0,0.000000,0.0,1.000000,0.0,39,23
2,135.6177,425.00,373.646250,1038.02000,3161.59,0,2,0.500000,0.000000,0.5,0.000000,0.0,0.0,0.000000,1.0,0.000000,0.0,16,54
3,1428.3333,1712.00,1272.388889,1582.00000,183224.00,1,2,0.000000,1.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.000000,0.0,144,39
4,1435.5716,2325.96,1026.027143,2372.00000,136461.61,0,2,0.142857,0.842105,0.0,0.015038,0.0,0.0,0.983607,0.0,0.016393,0.0,132,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3089374,425.0300,425.03,425.030000,425.03000,425.03,0,2,1.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.0,0.000000,0.0,1,22
3089375,435.7000,435.70,435.700000,435.70000,435.70,0,2,1.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.0,0.000000,0.0,1,20
3089376,167.9100,167.91,167.910000,167.91000,167.91,0,2,NaN,NaN,NaN,NaN,NaN,1.0,0.000000,0.0,0.000000,0.0,0,26
3089377,130.8000,130.80,130.800000,130.80000,130.80,1,2,NaN,NaN,NaN,NaN,NaN,1.0,0.000000,0.0,0.000000,0.0,0,25


In [13]:
modelo.predict_proba(data_otr)

array([[0.03519499, 0.964805  ],
       [0.99623984, 0.00376018],
       [0.9783673 , 0.02163269],
       ...,
       [0.3468731 , 0.6531269 ],
       [0.72921234, 0.27078766],
       [0.6101381 , 0.38986188]], dtype=float32)

In [14]:
modelo.predict(data_otr).sum()

1755055

In [15]:
1755055/3089379

0.568093134574942